# **Modelo estatístico com dados públicos do PySUS**

## Instalando bibliotecas e funções

---


In [1]:
!pip install pandas
!pip install numpy
!pip install matplotlib.pyplot
!pip install matplotlib.ticker
!pip install pylab
!pip install chart_studio
!pip install plotly.graph_objs
!pip install rpy2.robjects
!pip install seaborn
!pip install statsmodels.formula.api
!pip install ftplib
!pip install pysus.online_data.SIH
!pip install scipy.stats
!pip install sklearn.preprocessing
!pip install sklearn.metrics
!pip install sklearn.model_selection

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement matplotlib.ticker (from versions: none)
ERROR: No matching distribution found for matplotlib.ticker
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pylab (from versions: none)
ERROR: No matching distribution found for pylab
Looking in indexes: https://pypi.org/sim

 Importando bibliotecas e funções

In [2]:
# Obtenção de dados
from ftplib import FTP
#from pysus.online_data.SIH import download

# Manipulação de dados
import pandas as pd
import numpy as np

# Plotagem de gráficos
import pylab as pl
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import chart_studio.plotly as py
import rpy2.robjects as robjects
import plotly.graph_objs as gp

# Métricas estatísticas
import statsmodels.formula.api as smf
from sklearn.metrics import roc_curve, auc, precision_recall_fscore_support, classification_report

# Construção do modelo
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

## **Coleta da base de dados**

Importando base de dados

In [4]:
columns = ["N_AIH","IDADE","SEXO","UF_ZI","NUM_FILHOS","INSTRU","MORTE",
           "CID_MORTE","DIAS_PERM","DIAG_PRINC","DIAG_SECUN","DT_INTER",
           "ANO_CMPT","MES_CMPT","MARCA_UTI","UTI_MES_TO","INFEHOSP"]

df_dengue = pd.read_csv("/content/sih_dengue.csv")
df_dengue.set_axis(columns, axis = 1, inplace = True)
df_dengue.head()

,N_AIH,IDADE,SEXO,UF_ZI,NUM_FILHOS,INSTRU,MORTE,CID_MORTE,DIAS_PERM,DIAG_PRINC,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO,INFEHOSP
0,1219100035571,15,1,120000,0,0,0,0,3,A90,0,20190101,2019,1,0,0,NaN
1,1219100035615,22,1,120000,0,0,0,0,6,A90,0,20190104,2019,1,0,0,NaN
2,1218100569642,25,3,120000,0,0,0,0,4,A90,0,20181231,2019,1,0,0,NaN
3,1218100585350,27,1,120000,0,0,0,0,1,A90,0,20181216,2019,1,0,0,NaN
4,1219100036957,6,1,120000,0,0,0,0,8,A90,0,20190102,2019,1,0,0,NaN


## **EDA**

### Entendendo os tipos de variáveis existentes

In [5]:
display(df_dengue.info())
print()
df_dengue.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199555 entries, 0 to 199554
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   N_AIH       199555 non-null  int64  
 1   IDADE       199555 non-null  int64  
 2   SEXO        199555 non-null  int64  
 3   UF_ZI       199555 non-null  int64  
 4   NUM_FILHOS  199555 non-null  int64  
 5   INSTRU      199555 non-null  int64  
 6   MORTE       199555 non-null  int64  
 7   CID_MORTE   199555 non-null  int64  
 8   DIAS_PERM   199555 non-null  int64  
 9   DIAG_PRINC  199555 non-null  object 
 10  DIAG_SECUN  199555 non-null  int64  
 11  DT_INTER    199555 non-null  int64  
 12  ANO_CMPT    199555 non-null  int64  
 13  MES_CMPT    199555 non-null  int64  
 14  MARCA_UTI   199555 non-null  int64  
 15  UTI_MES_TO  199555 non-null  int64  
 16  INFEHOSP    0 non-null       float64
dtypes: float64(1), int64(15), object(1)
memory usage: 25.9+ MB


None

,N_AIH,IDADE,SEXO,UF_ZI,NUM_FILHOS,INSTRU,MORTE,CID_MORTE,DIAS_PERM,DIAG_PRINC,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO,INFEHOSP
0,1219100035571,15,1,120000,0,0,0,0,3,A90,0,20190101,2019,1,0,0,NaN
1,1219100035615,22,1,120000,0,0,0,0,6,A90,0,20190104,2019,1,0,0,NaN
2,1218100569642,25,3,120000,0,0,0,0,4,A90,0,20181231,2019,1,0,0,NaN
3,1218100585350,27,1,120000,0,0,0,0,1,A90,0,20181216,2019,1,0,0,NaN
4,1219100036957,6,1,120000,0,0,0,0,8,A90,0,20190102,2019,1,0,0,NaN


Com base nas informações acima, pode-se inferir que:

1. A variável N_AIH está formatada como int64. Porém, por se tratar de uma informação categórica, o correto seria str (string)

2. A coluna UF_ZI se refere à localidade e, pelo mesmo motivo acima, devia ser considerada do tipo string

### Explorando os valores de cada variável

In [6]:
for column in df_dengue.columns:
    print(f"A coluna {column} possui {df_dengue[column].nunique()} valores distintos, sendo: {df_dengue[column].unique()}")
    print()

A coluna N_AIH possui 99778 valores distintos, sendo: [1219100035571 1219100035615 1218100569642 ... 1721101767077 1721101767231
 1218100582215]

A coluna IDADE possui 100 valores distintos, sendo: [15 22 25 27  6  1 31 40 19 67 32 17 24 42 21 20  5 44 46 61 63 86 30 76
 55 39 48 33 83 58 52  9 56 11 64 26 43 53 60 16 29  3 45 34 36 57 75 59
 54 69  2 23 35 14 37 12 47 62 41 74 50 65 10 78 70 18 71  4 38 91 80  7
  8 13 73 87 51 84 77 66 72 68 28 49 92 79 90 81 89 82 94 85  0 95 97 88
 93 98 96 99]

A coluna SEXO possui 2 valores distintos, sendo: [1 3]

A coluna UF_ZI possui 1602 valores distintos, sendo: [120000 270000 270230 ... 150775 261120 241440]

A coluna NUM_FILHOS possui 1 valores distintos, sendo: [0]

A coluna INSTRU possui 1 valores distintos, sendo: [0]

A coluna MORTE possui 2 valores distintos, sendo: [0 1]

A coluna CID_MORTE possui 1 valores distintos, sendo: [0]

A coluna DIAS_PERM possui 65 valores distintos, sendo: [  3   6   4   1   8   7   2   5   9   0  11  10  

Pode-se analisar os seguintes pontos do código acima:

1. A coluna SEXO se refere a somente dois valores, sendo um para "masculino" (1) e outro "feminino" (3)
2. Todos os registros estudados não possuem filhos (NUM_FILHOS), podendo essa ser uma informação descartável do modelo
3. As variáveis INTRU e  CID_MORTE também possuem somente um valor, podendo ser mais  informações descartadas no momento
4. O diagnóstico principal (DIAG_PRIN) sempre vai ser A90 ou A900, pois a coleta foi feita somente com esses dados. Então, essa seria mais uma informação descartável
5. Não possuem valores distintos de 0 em DIAG_SECU, podendo essa ser mais uma informação descartável
6. A coluna INFEHOSP só possui valores NAs

### Verificando Missing Values

In [7]:
df_dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199555 entries, 0 to 199554
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   N_AIH       199555 non-null  int64  
 1   IDADE       199555 non-null  int64  
 2   SEXO        199555 non-null  int64  
 3   UF_ZI       199555 non-null  int64  
 4   NUM_FILHOS  199555 non-null  int64  
 5   INSTRU      199555 non-null  int64  
 6   MORTE       199555 non-null  int64  
 7   CID_MORTE   199555 non-null  int64  
 8   DIAS_PERM   199555 non-null  int64  
 9   DIAG_PRINC  199555 non-null  object 
 10  DIAG_SECUN  199555 non-null  int64  
 11  DT_INTER    199555 non-null  int64  
 12  ANO_CMPT    199555 non-null  int64  
 13  MES_CMPT    199555 non-null  int64  
 14  MARCA_UTI   199555 non-null  int64  
 15  UTI_MES_TO  199555 non-null  int64  
 16  INFEHOSP    0 non-null       float64
dtypes: float64(1), int64(15), object(1)
memory usage: 25.9+ MB


Observe a terceira coluna acima, ela representa a contagem de valores não nulos do dataframe.

Logo, nota-se que:

1. A coluna de INFEHOSP é composta somente de valores nulos, como mostrado também abaixo:

In [8]:
df_dengue.isna().sum()

N_AIH              0
IDADE              0
SEXO               0
UF_ZI              0
NUM_FILHOS         0
INSTRU             0
MORTE              0
CID_MORTE          0
DIAS_PERM          0
DIAG_PRINC         0
DIAG_SECUN         0
DT_INTER           0
ANO_CMPT           0
MES_CMPT           0
MARCA_UTI          0
UTI_MES_TO         0
INFEHOSP      199555
dtype: int64

## Tratamento dos dados

### Corrigindo as tipagens das variáveis

In [9]:
df_dengue["N_AIH"] = df_dengue["N_AIH"].astype(str)
df_dengue["UF_ZI"] = df_dengue["UF_ZI"].astype(str)
df_dengue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199555 entries, 0 to 199554
Data columns (total 17 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   N_AIH       199555 non-null  object 
 1   IDADE       199555 non-null  int64  
 2   SEXO        199555 non-null  int64  
 3   UF_ZI       199555 non-null  object 
 4   NUM_FILHOS  199555 non-null  int64  
 5   INSTRU      199555 non-null  int64  
 6   MORTE       199555 non-null  int64  
 7   CID_MORTE   199555 non-null  int64  
 8   DIAS_PERM   199555 non-null  int64  
 9   DIAG_PRINC  199555 non-null  object 
 10  DIAG_SECUN  199555 non-null  int64  
 11  DT_INTER    199555 non-null  int64  
 12  ANO_CMPT    199555 non-null  int64  
 13  MES_CMPT    199555 non-null  int64  
 14  MARCA_UTI   199555 non-null  int64  
 15  UTI_MES_TO  199555 non-null  int64  
 16  INFEHOSP    0 non-null       float64
dtypes: float64(1), int64(13), object(3)
memory usage: 25.9+ MB


### Excluindo valores redundantes

In [10]:
df_dengue.drop(["NUM_FILHOS", "CID_MORTE", "DIAG_PRINC", "INSTRU", "DIAG_PRINC"], axis = 1, inplace = True)
df_dengue.head()

,N_AIH,IDADE,SEXO,UF_ZI,MORTE,DIAS_PERM,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO,INFEHOSP
0,1219100035571,15,1,120000,0,3,0,20190101,2019,1,0,0,NaN
1,1219100035615,22,1,120000,0,6,0,20190104,2019,1,0,0,NaN
2,1218100569642,25,3,120000,0,4,0,20181231,2019,1,0,0,NaN
3,1218100585350,27,1,120000,0,1,0,20181216,2019,1,0,0,NaN
4,1219100036957,6,1,120000,0,8,0,20190102,2019,1,0,0,NaN


### Excluindo missing values

In [11]:
df_dengue.drop("INFEHOSP", axis = 1, inplace = True)
df_dengue.head()

,N_AIH,IDADE,SEXO,UF_ZI,MORTE,DIAS_PERM,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO
0,1219100035571,15,1,120000,0,3,0,20190101,2019,1,0,0
1,1219100035615,22,1,120000,0,6,0,20190104,2019,1,0,0
2,1218100569642,25,3,120000,0,4,0,20181231,2019,1,0,0
3,1218100585350,27,1,120000,0,1,0,20181216,2019,1,0,0
4,1219100036957,6,1,120000,0,8,0,20190102,2019,1,0,0


### Atribuindo o número do diagnóstico ao índice

In [12]:
df_dengue.index = df_dengue["N_AIH"]
df_dengue.drop("N_AIH", axis = 1, inplace = True)
df_dengue.head()

,IDADE,SEXO,UF_ZI,MORTE,DIAS_PERM,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO
N_AIH,,,,,,,,,,,
1219100035571,15,1,120000,0,3,0,20190101,2019,1,0,0
1219100035615,22,1,120000,0,6,0,20190104,2019,1,0,0
1218100569642,25,3,120000,0,4,0,20181231,2019,1,0,0
1218100585350,27,1,120000,0,1,0,20181216,2019,1,0,0
1219100036957,6,1,120000,0,8,0,20190102,2019,1,0,0


## **Feature Engineering**

### Atribuindo termos à variável SEXO

Para renomear a variável SEXO, é necessário saber que 1 se refere à Masculino e 3 à feminino

In [14]:
sexo = {1: "Masculino", 3: "Feminino"}

df_dengue["SEXO"] = df_dengue["SEXO"].replace(sexo)
df_dengue.head()

,IDADE,SEXO,MORTE,DIAS_PERM,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO,REGIAO
N_AIH,,,,,,,,,,,
1219100035571,15,Masculino,0,3,0,20190101,2019,1,0,0,Norte
1219100035615,22,Masculino,0,6,0,20190104,2019,1,0,0,Norte
1218100569642,25,Feminino,0,4,0,20181231,2019,1,0,0,Norte
1218100585350,27,Masculino,0,1,0,20181216,2019,1,0,0,Norte
1219100036957,6,Masculino,0,8,0,20190102,2019,1,0,0,Norte


### Criando a variável REGIAO a partir da UF_ZI

Para fazer isso, é necessário saber que o primeiro dígito do UF_ZI se refere à região do indivíduo. Logo, basta fazer o fatiamento desse valor e associar à região.

In [13]:
# Para retirar o primeiro dígito, basta fazer o slicing entre 0 (inclusivo) e 1 (exclusivo)
df_dengue["REGIAO"] = df_dengue["UF_ZI"].str.slice(0, 1)
print(f"Valores distintos para a variável REGIAO: {df_dengue['REGIAO'].unique()}")
print()

# Com base no dígito recolhido, basta associar ele com a região disposta no dicionário abaixo:
regiao = {"1":"Norte", "2":"Nordeste", "3":"Sudeste", "4":"Sul", "5":"Centro-Oeste"}

# Por meio da função .replace()
df_dengue["REGIAO"] = df_dengue["REGIAO"].replace(regiao)
print(f"Valores distintos para a variável REGIAO: {df_dengue['REGIAO'].unique()}")
print()

# Excluindo a variável UF_ZI
df_dengue.drop("UF_ZI", axis = 1, inplace = True)

df_dengue.head()

Valores distintos para a variável REGIAO: ['1' '2' '5' '3' '4']

Valores distintos para a variável REGIAO: ['Norte' 'Nordeste' 'Centro-Oeste' 'Sudeste' 'Sul']



,IDADE,SEXO,MORTE,DIAS_PERM,DIAG_SECUN,DT_INTER,ANO_CMPT,MES_CMPT,MARCA_UTI,UTI_MES_TO,REGIAO
N_AIH,,,,,,,,,,,
1219100035571,15,1,0,3,0,20190101,2019,1,0,0,Norte
1219100035615,22,1,0,6,0,20190104,2019,1,0,0,Norte
1218100569642,25,3,0,4,0,20181231,2019,1,0,0,Norte
1218100585350,27,1,0,1,0,20181216,2019,1,0,0,Norte
1219100036957,6,1,0,8,0,20190102,2019,1,0,0,Norte
